In [2]:
import torch
from transformers import (
    AdamW,
    AutoModel,
    get_linear_schedule_with_warmup,
    AutoTokenizer,
    AutoConfig
)

In [4]:
import sys
module_path = '/home/x1112436/git/sent-semantic-repo'
sys.path.append(module_path)

In [6]:
from src.utils import set_seed
from src.trainer import SimcseTrainer
from src.dataset import DATASET_MAPPING_DICT
from src.utils import PreprocessorFactory 
from src.utils import get_model_argparse
from src.model import MODEL_MAPPING_DICT
from src.model import CONFIG_MAPPING_DICT
from src.logger import Experi_Logger
from config.nli_config import nli_parser_model_args

In [7]:
args = nli_parser_model_args()

In [8]:
#args.pretrained_model = '/home/x1112436/model_file/faq_sent_roberta/sent_roberta'
#args.pretrained_model = f'/home/x1112436/result/faq/modelfile/{args.pretrained_model}'
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.model_max_len = 100
args.is_preprocessed = True
args.valid_first = False
args.data_type='triple'
args.loss= 'TripletLoss'
args.margin = 1.0

In [83]:
#args.pretrained_model_final = '/home/x1112436/result/faq/modelfile/home/x1112436/model_file/sent_roberta'
#args.pretrained_model = '/home/x1112436/result/faq/modelfile2/klue/roberta-large'

In [84]:
model = MODEL_MAPPING_DICT['sent_roberta'].from_pretrained(
    args.pretrained_model, **vars(args), 
)
tokenizer = AutoTokenizer.from_pretrained(args.pretrained_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# LOAD DATA

In [1]:
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client

In [6]:
dataset = 'x1112436'
log_table = 'faq_table'
query = f"""

SELECT  TRIM(query) as query,
        answer,
        REPLACE(TRIM(intent_nm), "'", "") as intent_nm,
        answer,
        domain,
        status
FROM `skt-datahub.{dataset}.{log_table}`
WHERE intent_nm !='' and intent_nm is not null
"""

In [7]:
faq_table = bq_to_pandas(query)

query: 

SELECT  TRIM(query) as query,
        answer,
        REPLACE(TRIM(intent_nm), "'", "") as intent_nm,
        answer,
        domain,
        status
FROM `skt-datahub.x1112436.faq_table`
WHERE intent_nm !='' and intent_nm is not null

destination: skt-datahub._775c5ccab1096b3cccd7ac34a5db11c0a354fb07.anon2441652b26c102a5fa984effca804675c71dde12e9edc20962cb33cf9629c4d1
total_rows: 246427
slot_secs: 0.971

Downloading: 100%|██████████|


In [9]:
faq_table.head(2)

,query,answer,intent_nm,answer_1,domain,status
0,T RING,'T Ring(티 링)'에 대해 안내해 드릴게요.\nT Ring은 SK텔레콤의 상징...,T Ring,'T Ring(티 링)'에 대해 안내해 드릴게요.\nT Ring은 SK텔레콤의 상징...,sms_customer_center,INSERT
1,슬림,'슬림 요금제'에 대해 안내해 드릴게요.\n슬림 요금제는 5G 스마트폰을 사용하는 ...,슬림,'슬림 요금제'에 대해 안내해 드릴게요.\n슬림 요금제는 5G 스마트폰을 사용하는 ...,sms_customer_center,INSERT


In [21]:
# intent_nm_ans = faq_table[['intent_nm', 'answer']].drop_duplicates(keep='first')
# intent_nm_ans.to_csv('./result/intent_nm.csv', encoding='utf-8-sig', index=False)

In [22]:
idx2query = list(faq_table['query'].unique())
idx2intent_nm = list(faq_table.intent_nm.unique())

In [24]:
# faq_table_q_a = faq_table[['qry_txt_cont', 'intent_nm']]
# query_to_answer = faq_table_q_a.set_index('qry_txt_cont').to_dict()['intent_nm']

# INPUT for single embedding

In [31]:
from torch.utils.data import (
    DataLoader, Dataset
)

In [32]:
from dataclasses import dataclass
from typing import List, Any, Union, Dict

In [33]:
@dataclass
class SingleSentenceInput:
    sentence_a: str = None
    a_input_ids: List[int] = None
    a_attention_mask: List[int] = None

In [34]:
class EmbeddingDataset(Dataset):
    def __init__(
            self,
            args,
            features:List[SingleSentenceInput],
            max_length,
            tokenizer,
            **kwargs
    ):
        super(EmbeddingDataset, self).__init__()
        self.args = args
        self.features = features
        self.max_length = max_length
        self.pad_token_id = tokenizer.pad_token_id
        self.sep_token_id = tokenizer.sep_token_id if tokenizer.sep_token_id else tokenizer.eos_token_id

    def __getitem__(self, index) -> Dict[str, Any]:
        feature = self.features[index]
        return {
            'a_sentence': feature.sentence_a,
            'a_input_ids': torch.tensor(feature.a_input_ids, dtype=torch.long),
            'a_attention_mask': torch.tensor(feature.a_attention_mask, dtype=torch.long)
        }
    def __len__(self):
        return len(self.features)
    
    def loader(self, shuffle:bool=True, batch_size:int=64):
        return DataLoader(self, shuffle=shuffle, batch_size=batch_size, collate_fn=self.collater)

    def collater(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:

        a_sentence = [data['a_sentence'] for data in batch]
        a_input_ids = [data['a_input_ids'] for data in batch]
        a_attention_mask = [data['a_attention_mask'] for data in batch]
        ##  token level encoding
        batch_size = len(batch)
        sizes = [len(s) for s in a_input_ids]
        target_size = min(max(sizes), self.max_length)
        """ torch.full -> creates a tensor of a given shape and fills it with a scalar value self.pad_token_id here"""
        a_collated_ids = torch.full((batch_size, target_size), self.pad_token_id, dtype=torch.long)
        a_collated_attention_masks = torch.zeros((batch_size, target_size), dtype=torch.long)

        """ cut data if size > target_size else: fill by self.pad_token_id """
        for i, (input_id, attention_m, size) in enumerate(
                zip(a_input_ids, a_attention_mask, sizes)):
            diff = target_size - size
            if diff < 0:
                a_collated_ids[i, :target_size] = input_id[:target_size]
                a_collated_ids[i, -1] = self.sep_token_id
                a_collated_attention_masks[i, :target_size] = attention_m[:target_size]

            else:
                a_collated_ids[i, :size] = input_id
                a_collated_attention_masks[i, :size] = attention_m

        return {
            'a_sentence': a_sentence,
            'a_input_ids': a_collated_ids,
            'a_attention_mask': a_collated_attention_masks
        }

In [35]:
from src.utils.abs_preprocess import AbsPreprocessor

class Testprocessor(AbsPreprocessor):

    @classmethod
    def preprocess(cls, tokenizer,  input_list:List) -> None:
        """ try read tsv file using pandas first if [memory or parse] error catched use other reading method  """
    
        feature_list = list()
        skipped_line = 0

        for i, line in enumerate(input_list):
            try:
                a_encoded_sentence = cls.tokenizing(input=line, tokenizer=tokenizer, tokenizer_input=None)
                feature_list.append(
                    SingleSentenceInput(
                        sentence_a = line,
                        a_input_ids = a_encoded_sentence.input_ids,
                        a_attention_mask=a_encoded_sentence.attention_mask,
                    )
                )
            except Exception as e:
                print(f'Error occurs in {i} lines in preprocessing')
                print(line)
                print(e)
                break

        return feature_list

In [39]:
query_input = Testprocessor.preprocess(tokenizer = tokenizer ,input_list = idx2query)

In [37]:
intent_input = Testprocessor.preprocess(tokenizer = tokenizer ,input_list = idx2intent_nm)

In [40]:
queryDataset = EmbeddingDataset(args=args, features=query_input, max_length=args.model_max_len, tokenizer=tokenizer)

In [41]:
intentDataset = EmbeddingDataset(args=args, features=intent_input, max_length=args.model_max_len, tokenizer=tokenizer)

# DataLoader

In [44]:
queryDataloader = queryDataset.loader(
            shuffle=False, batch_size=400 )

In [45]:
intentDataloader = intentDataset.loader(
            shuffle=False, batch_size=400 )

# embed query and intent

In [47]:
from tqdm.notebook import tqdm

In [85]:
model = model.to(args.device)

In [ ]:
#next(iter(query_dataloader))

In [86]:
# model.eval()
# query_list = []
# embedding_query = []
# with torch.no_grad():   
#     for batch_idx, batch in enumerate(tqdm(query_dataloader)): 
#         batch = {key: (item.to(args.device) if type(item) == torch.Tensor else item) for key, item in batch.items()}
#         a_embedding = model(batch['a_input_ids'], batch['a_attention_mask'])
#         a_sentence = batch['a_sentence']
#         query_list.extend(a_sentence)
#         embedding_query.append(a_embedding)
#     embedding_query = torch.cat(embedding_query, 0) 

In [87]:
model.eval()
intent_list = []
embedding_intent = []
with torch.no_grad():   
    for batch_idx, batch in enumerate(tqdm(intent_dataloader)): 
        batch = {key: (item.to(args.device) if type(item) == torch.Tensor else item) for key, item in batch.items()}
        a_embedding = model(batch['a_input_ids'], batch['a_attention_mask'])
        a_sentence = batch['a_sentence']
        intent_list.extend(a_sentence)
        embedding_intent.append(a_embedding)
    embedding_intent = torch.cat(embedding_intent, 0) 


  0%|          | 0/5 [00:00<?, ?it/s]

In [88]:
model.eval()
service_query_list = []
embedding_service_query = []
with torch.no_grad():   
    for batch_idx, batch in enumerate(tqdm(service_query_dataloader)): 
        batch = {key: (item.to(args.device) if type(item) == torch.Tensor else item) for key, item in batch.items()}
        a_embedding = model(batch['a_input_ids'], batch['a_attention_mask'])
        a_sentence = batch['a_sentence']
        service_query_list.extend(a_sentence)
        embedding_service_query.append(a_embedding)
    embedding_service_query = torch.cat(embedding_service_query, 0) 


  0%|          | 0/17 [00:00<?, ?it/s]

In [89]:
import torch.nn.functional as F
values, indices = torch.topk(F.normalize(embedding_service_query, dim =1) @ F.normalize(embedding_intent, dim=1).T, 3)

In [90]:
threshold = 0.4

In [91]:
values[values < threshold] = -1
indices[values < threshold] = -1

In [92]:
indices = indices.cpu().numpy()

In [93]:
predict_dict = dict()
for i in range(embedding_service_query.size()[0]):        
    predict_dict[service_queries[i]] = [(intent_list[indices[i, j]], round(values[i,j].cpu().item(), 3)) for j in range(3) if indices[i,j] != -1]

predict_list = []
for key, value in predict_dict.items():
    query = key
    for i in range(3 - len(value)):
        value.append('')
        
    predict_list.append({'query': query, 'n_rank1': value[0], 'n_rank2': value[1], 'n_rank3': value[2]})
            

In [94]:
predict_pd = pd.DataFrame(predict_list)

In [95]:
join_data = pd.merge(predict_pd, service_queries_pd, on = 'query')

In [96]:
join_data.to_csv('../result/result_v1.csv', encoding='utf-8-sig', index=False)

# sampling for result v1 v2

In [98]:
result_1 = pd.read_csv('../result/result_v1.csv')

In [111]:
result_2 = pd.read_csv('../result/result_v2.csv')

In [100]:
result_1.head(3)

,query,n_rank1,n_rank2,n_rank3,count,p_rank1,p_rank2,p_rank3
0,미납요금 납부 가능일 문의,"('SKT 미납센터 연락처', 0.884)","('미납 이용정지', 0.874)","('과납', 0.858)",49194,SKT미납센터연락처(0.4809759),미납이용정지(0.457217725),미납직권해지(0.43127817)
1,미납 문의할게,"('SKT 미납센터 연락처', 0.97)","('미납 이용정지', 0.935)","('과납', 0.881)",27234,SKT해지미납센터연락처(0.4995),SKT미납센터연락처(0.4940547),미납직권해지(0.4642235)
2,미납문의할게,"('SKT 미납센터 연락처', 0.956)","('미납 이용정지', 0.903)","('과납', 0.9)",1616,SKT미납센터연락처(0.49495625),미납직권해지(0.463400065),미납이용정지(0.4573068)


In [112]:
result_2 = result_2.rename(columns={'n_rank1': 'f_rank1', 'n_rank2': 'f_rank2','n_rank3': 'f_rank3'})

In [114]:
result_2 = result_2[['query', 'f_rank1', 'f_rank2', 'f_rank3']]

In [116]:
final_result = pd.merge(result_1, result_2, on = ['query'], how='left')

In [118]:
final_result['count'] = final_result['count'].astype('int')

In [120]:
final_result[final_result['count'] > 1]

,query,n_rank1,n_rank2,n_rank3,count,p_rank1,p_rank2,p_rank3,f_rank1,f_rank2,f_rank3
0,미납요금 납부 가능일 문의,"('SKT 미납센터 연락처', 0.884)","('미납 이용정지', 0.874)","('과납', 0.858)",49194,SKT미납센터연락처(0.4809759),미납이용정지(0.457217725),미납직권해지(0.43127817),"('요금 납부일 확인 방법', 0.754)","('요금 납부일 변경 방법', 0.607)","('SKT 미납센터 연락처', 0.508)"
1,미납 문의할게,"('SKT 미납센터 연락처', 0.97)","('미납 이용정지', 0.935)","('과납', 0.881)",27234,SKT해지미납센터연락처(0.4995),SKT미납센터연락처(0.4940547),미납직권해지(0.4642235),"('SKT 미납센터 연락처', 0.734)","('과납', 0.672)","('미납 이용정지', 0.664)"
2,미납문의할게,"('SKT 미납센터 연락처', 0.956)","('미납 이용정지', 0.903)","('과납', 0.9)",1616,SKT미납센터연락처(0.49495625),미납직권해지(0.463400065),미납이용정지(0.4573068),"('SKT 미납센터 연락처', 0.757)","('미납 이용정지', 0.654)","('과납', 0.634)"
3,데이터 자동선물 신청할래,"('자동연결 유료 서비스', 0.989)","('자동연결 유료 서비스 신청 방법', 0.988)","('가족 간 데이터 자동 선물 발송 시점', 0.988)",1315,가족간데이터자동선물하기(0.48386545),T끼리데이터선물하기비용(0.48172125),T끼리데이터선물하기대상(0.4792024),"('가족 간 데이터 자동 선물하기', 0.621)","('T끼리 데이터 선물 조르기 방법', 0.574)","('가족 간 데이터 자동 선물 발송 시점', 0.544)"
4,로밍 요금제 한눈에 보기,"('B tv 요금제', 0.938)","('요금제문의', 0.932)","('유선 인터넷 요금제', 0.931)",1125,로밍요금확인시점(0.49537815),로밍통화요금(0.49452138),로밍수신요금(0.4935397),"('로밍 요금 확인 시점', 0.535)","('T로밍 Biz 요금제', 0.512)","('원패스 제공 국가', 0.405)"
...,...,...,...,...,...,...,...,...,...,...,...
4420,시아야,"('\u200bT아이폰케어 SE 2022', 0.749)","('A.', 0.748)","('퀵보이스', 0.728)",2,팅요금제자동변경시기(0.42258522),가입정보조회(0.41399312),시니어요금제(0.40805438),"('세이프캐시', 0.434)",NaN,NaN
4421,영화 볼레,"('롯데 TELLO 카드', 0.469)","('휴대폰 렌탈', 0.467)","('휴대폰 렌탈 예약 방법', 0.449)",2,T멤버십VIPPick영화예매방법(0.48877975),VIP요금제프로그램(0.452621075),T멤버십VIPPick잔여횟수확인방법(0.401124475)\n,"('V 컬러링', 0.403)",NaN,NaN
4422,T ALL케어 부가서비스 해지하고 싶습니다.,"('아파트청약케어 해지 방법', 0.961)","('예상 해지 금액', 0.928)","('가족 결합 상품 해지 방법', 0.926)",2,단말보험해지방법(0.416549475),단말보험가입방법(0.405288)\n,NaN,"('단말보험 해지 방법', 0.648)","('T 아이폰케어 해지 방법', 0.539)","('T All케어플러스 포인트 서비스 해지 방법', 0.533)"
4423,요금납부 전화번호 알고싶어요,"('요금 부분 납부 방법', 0.885)","('과납', 0.858)","('미납 직권해지', 0.764)",2,요금수납내역확인방법(0.47446915),요금부분납부방법(0.46495125),기타요금(0.46109585),"('SKT 미납센터 연락처', 0.567)","('자동납부번호 확인 방법', 0.538)","('요금 수납 내역 확인 방법', 0.486)"


In [121]:
final_result['count'].unique()

array([49194, 27234,  1616,  1315,  1125,  1005,   746,   667,   523,
         252,   223,   198,   188,   180,   178,   172,   169,   154,
         130,   116,   106,    83,    76,    65,    54,    52,    51,
          50,    44,    40,    39,    38,    36,    35,    34,    33,
          32,    30,    24,    22,    20,    19,    18,    17,    16,
          15,    14,    13,    12,    11,    10,     9,     8,     7,
           6,     5,     4,     3,     2,     1])

In [123]:
weighted_sample = final_result.sample(n=1100, weights="count")


In [127]:
weighted_sample = weighted_sample.sort_values(by=['count'], ascending=False)

In [130]:
weighted_sample.head(100)

,query,n_rank1,n_rank2,n_rank3,count,p_rank1,p_rank2,p_rank3,f_rank1,f_rank2,f_rank3
0,미납요금 납부 가능일 문의,"('SKT 미납센터 연락처', 0.884)","('미납 이용정지', 0.874)","('과납', 0.858)",49194,SKT미납센터연락처(0.4809759),미납이용정지(0.457217725),미납직권해지(0.43127817),"('요금 납부일 확인 방법', 0.754)","('요금 납부일 변경 방법', 0.607)","('SKT 미납센터 연락처', 0.508)"
1,미납 문의할게,"('SKT 미납센터 연락처', 0.97)","('미납 이용정지', 0.935)","('과납', 0.881)",27234,SKT해지미납센터연락처(0.4995),SKT미납센터연락처(0.4940547),미납직권해지(0.4642235),"('SKT 미납센터 연락처', 0.734)","('과납', 0.672)","('미납 이용정지', 0.664)"
2,미납문의할게,"('SKT 미납센터 연락처', 0.956)","('미납 이용정지', 0.903)","('과납', 0.9)",1616,SKT미납센터연락처(0.49495625),미납직권해지(0.463400065),미납이용정지(0.4573068),"('SKT 미납센터 연락처', 0.757)","('미납 이용정지', 0.654)","('과납', 0.634)"
3,데이터 자동선물 신청할래,"('자동연결 유료 서비스', 0.989)","('자동연결 유료 서비스 신청 방법', 0.988)","('가족 간 데이터 자동 선물 발송 시점', 0.988)",1315,가족간데이터자동선물하기(0.48386545),T끼리데이터선물하기비용(0.48172125),T끼리데이터선물하기대상(0.4792024),"('가족 간 데이터 자동 선물하기', 0.621)","('T끼리 데이터 선물 조르기 방법', 0.574)","('가족 간 데이터 자동 선물 발송 시점', 0.544)"
4,로밍 요금제 한눈에 보기,"('B tv 요금제', 0.938)","('요금제문의', 0.932)","('유선 인터넷 요금제', 0.931)",1125,로밍요금확인시점(0.49537815),로밍통화요금(0.49452138),로밍수신요금(0.4935397),"('로밍 요금 확인 시점', 0.535)","('T로밍 Biz 요금제', 0.512)","('원패스 제공 국가', 0.405)"
...,...,...,...,...,...,...,...,...,...,...,...
125,내요금제,"('요금제문의', 0.947)","('요금제 추천', 0.936)","('요금약정 할인제도', 0.931)",8,과납(0.42699795),부가사용금액(0.41011563)\n,NaN,"('요금제문의', 0.648)","('VIP 요금제 프로그램', 0.438)","('유선 인터넷 요금제', 0.428)"
108,01094203667 1기가 선물,"('T끼리 데이터 선물하기 연령', 0.799)","('T끼리 데이터 선물하기 대상', 0.769)","('T끼리 데이터 선물하기 알림', 0.757)",8,T끼리데이터선물하기받은데이터사용방법(0.42342803),T끼리데이터선물하기연령(0.42330955),T끼리데이터선물하기대상(0.416208),"('baro 1GB 충전', 0.403)",NaN,NaN
135,미납가능일,"('SKT 미납센터 연락처', 0.927)","('미납 이용정지', 0.909)","('과납', 0.866)",7,SKT미납센터연락처(0.48734835),미납이용정지(0.4625415),미납직권해지(0.43736097),"('요금 납부일 확인 방법', 0.642)","('가산금', 0.6)","('SKT 미납센터 연락처', 0.579)"
131,말일납부가능,"('요금 납부일 변경 방법', 0.892)","('요금 납부일 확인 방법', 0.877)","('유선 요금 납부일 변경 방법', 0.849)",7,요금납부일확인방법(0.9039121)\n,NaN,NaN,"('요금 납부일 변경 방법', 0.695)","('요금 납부일 확인 방법', 0.656)","('단말기 할부금 중도 납부 방법', 0.532)"


In [133]:
weighted_sample.to_csv('../result/final_reulst.csv', encoding='utf-8-sig', index=False)

In [147]:
dd = pd.read_csv('../result/result_tbl.csv', index_col=False)
dd.drop('Unnamed: 0', axis=1, inplace=True)
dd.drop('index', axis=1, inplace=True)

In [135]:
pd.merge(dd, 

1274

In [139]:
result_1 = result_1[['query', 'n_rank1', 'n_rank2', 'n_rank3']]

In [149]:
pd.merge(dd, result_1, how = on

,query,f_rank1,f_rank2,f_rank3
0,미납요금 납부 가능일 문의,"('요금 납부일 확인 방법', 0.754)","('요금 납부일 변경 방법', 0.607)","('SKT 미납센터 연락처', 0.508)"
1,미납 문의할게,"('SKT 미납센터 연락처', 0.734)","('과납', 0.672)","('미납 이용정지', 0.664)"
2,미납문의할게,"('SKT 미납센터 연락처', 0.757)","('미납 이용정지', 0.654)","('과납', 0.634)"


In [148]:
dd.head(3)

,query,query_cnt,p_rank1,p_rank2,p_rank3,n_rank_1,n_rank2,n_rank3
0,미납요금 납부 가능일 문의,49194,SKT미납센터연락처,미납이용정지,미납직권해지,요금납부일확인방법,SKT미납센터연락처,미납이용정지
1,미납 문의할게,27234,SKT해지미납센터연락처,SKT미납센터연락처,미납직권해지,SKT미납센터연락처,미납이용정지,과납
2,로밍 요금제 한눈에 보기,1125,로밍요금확인시점,로밍통화요금,로밍수신요금,NaN,NaN,NaN
